In [1]:
import yfinance as yf
import pandas as pd
import pprint
from tqdm.notebook import tqdm
import matplotlib.pyplot as plot
import seaborn as sns
import pickle

## Importazione della lista di stock supportata da revolut

In [2]:
stocks_list = pd.read_csv('Stock_list.csv', sep=';')
stocks = ''
for item in stocks_list['Ticker']:
    stocks = stocks + ' ' + str(item)

In [4]:
stocks

' NVDA AMD PACB DARD CORS CRIS METT SATI ELAN GNTX ORGN ISRG XPEV CRSR CRSP SIRI ZBRA CWEN SWKS FFIV AMRC OTLY CROX PLNT CTXS SPLK MGIC FAST MARA AFRM ARCT MVIS ARCC PENN KEYS CTVA BLNK UNIT NPTN BEPC MANU FMAC YETI TALK CDNS ATIP CTSH SUNP BBVA MDRX AGNC SPWR LEVI DELL CARS FSLR BBAR AMAT CEME IRBT DDOG SLCA FVRR PTON ADSK REGI REGN ARMO ALLY VRTX COST BRFS HIMX ANGI ORCL LULU KRAT BARK DIDI ASAN PDCE POSH TPGY CSIQ VIAC FRSH HYZN UCTT CURI TROW CCCS APSG CSGP KTOS NVCR PATH LESL JNPR LCID MRNA FROG GOEV NVAX APPN RDFN EXAS TDOC TMUS VRSN BTAQ ELMS SCCO ARVL ULTA ROST CLBT BMBL ISIZ BYND SPCE NCLH GILD UBER NVDA BKNG ATVI GOOG GRPN AMRX INTC SBUX ABEV SHOP GPRO ADBE CSCO FOXA ABBV TWTR APPS SNAP QCOM PYPL ZNGA JBLU NYMT PINS OMAR SEAB INIC OSCR BMRN MNDT EQNR NTAP GMED FUBO CERN HSBC TIMB SLVM SSSS RKLB DISC PAYX PAYO TPIC MIME CLNE DKNG JMIA COTY ENLC MTDR BSBR LXRX SPOT PLUG ARNC TEVA TRIP GOLD EBAY BIDU DOCU TWOU EXPE SPGI ROKU RACE LYFT IMGN NORD JIUD FWON FTNT TRGP TTWO INFY MTCH

### Scaricamento dei dati di storico delle aziende

In [5]:
data = yf.download(  # or pdr.get_data_yahoo(...
        # tickers list or string as well
        tickers = stocks[:10],

        # use "period" instead of start/end
        # valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
        # (optional, default is '1mo')
        period = "2y",

        # fetch data by interval (including intraday if period < 60 days)
        # valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
        # (optional, default is '1d')
        interval = "1d",

        # group by ticker (to access via data['SPY'])
        # (optional, default is 'column')
        group_by = 'column',

        # adjust all OHLC automatically
        # (optional, default is False)
        auto_adjust = False,

        # download pre/post regular market hours data
        # (optional, default is False)
        prepost = False,

        # use threads for mass downloading? (True/False/Integer)
        # (optional, default is True)
        threads = True,

        # proxy URL scheme use use when downloading?
        # (optional, default is None)
        proxy = None
    )

data.to_pickle('dataset.pkl')

[*********************100%***********************]  2 of 2 completed


### Download dei dati delle aziende

In [ ]:
#ff = pd.DataFrame(yf.Ticker('MSFT').info)
#to_del = []
#
#for company in tqdm(pos_perf.index):
#    try:
#        ff = ff.append(yf.Ticker(company).info, ignore_index=True).fillna(0)
#    except Exception as e:
#        print(company)
#        to_del.append(company)
#        print(e)
#        continue

## Salvataggio dei dati scaricati

In [6]:
data = pd.read_pickle('dataset.pkl')

In [7]:
ff = pd.read_pickle('descriptions.pkl')

In [8]:
data.fillna(value=0, inplace=True)

## Analisi sulla performance delle 791 aziende nei 2 anni

In [9]:
def get_stock_data(df, stocksimbol):
    df = df.xs(stocksimbol, axis=1, level = 1, drop_level=True)
    df.drop(columns='Close', inplace=True)
    df.rename(columns = {'Adj Close':'Close'}, inplace=True)
    df.columns = df.columns.str.lower()
    return df

In [10]:
import pandas as pd
from stockstats import StockDataFrame as Sdf
from tqdm.notebook import tqdm

data = pd.read_pickle('dataset.pkl')
data = data.drop(data.index[0])

score = []
name = []
diff = []

for symbol in tqdm(data.columns.get_level_values(1).unique().values):
    try:
        stock  = Sdf.retype(get_stock_data(data, symbol))

        signal = stock['macds']        # Your signal line
        macd   = stock['macd']         # The MACD that need to cross the signal line
        #                                              to give you a Buy/Sell signal
        listLongShort = ["No data"]    # Since you need at least two days in the for loop

        for i in range(1, len(signal)):
            #                          # If the MACD crosses the signal line upward
            if macd[i] > signal[i] and macd[i - 1] <= signal[i - 1]:
                listLongShort.append(1)
            #                          # The other way around
            elif macd[i] < signal[i] and macd[i - 1] >= signal[i - 1]:
                listLongShort.append(-1)
            #                          # Do nothing if not crossed
            else:
                listLongShort.append(0)

        stock['Advice'] = listLongShort
        # The advice column means "Buy/Sell/Hold" at the end of this day or
        #  at the beginning of the next day, since the market will be closed
        stock['Advice'][0] = 0

        if stock[stock['Advice']!=0]['advice'][-1] == 1 and stock[stock['Advice']!=0]['advice'].index[-1] != stock['Advice'].index[-1]:
            stock['Advice'][-1] = -1

        elif stock[stock['Advice']!=0]['advice'][-1] == 1 and stock[stock['Advice']!=0]['advice'].index[-1] == stock['Advice'].index[-1]:
            #stock['Advice'][-2] = -1
            stock['Advice'][-1] = 0

        gain = stock['close']*stock['Advice']*(-1)
        delta = (stock['close'][-1]-stock['close'][0])/(stock['close'][0])
        #stock['gain'] = gain
        #print(gain.sum()/(stock['close'][-1]-stock['close'][0]))
        
        score.append(gain.sum()/(stock['close'][0]))
        name.append(symbol)
        diff.append(delta)
    except:
        continue

diff = pd.DataFrame(diff, index=name)
scores = pd.DataFrame(score, index=name)
scores.rename(columns={0:'Performance'}, inplace=True)

  0%|          | 0/2 [00:00<?, ?it/s]

/home/wonka/.local/lib/python3.10/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
/home/wonka/.local/lib/python3.10/site-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
/tmp/ipykernel_45654/2612653541.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stock['Advice'][0] = 0


In [11]:
ff = pd.read_pickle('descriptions.pkl')
ff = ff[['symbol', 'sector']]
ff.set_index('symbol', inplace=True)
scores = pd.merge(ff, scores, left_index=True, right_index=True)
scores = pd.merge(scores, diff, left_index=True, right_index=True)
scores['sector'].replace(0, 'Unknown', inplace=True)
scores.rename(columns={0:'diff'}, inplace=True)
scores.to_csv('scores.csv', sep=';', decimal=',')

In [12]:
#fig = plot.figure(figsize=(70, 10))
filtered = scores[scores['Performance']>1.50]
#sns.scatterplot(data=filtered.sort_values(by='sector'), x=filtered.index, y='Performance', hue='sector')
#plot.savefig('performance_sector.svg')
filtered.sort_values(by='Performance', ascending=False).to_csv('best_performers_MACD.csv', sep=';', decimal=',')

In [15]:
ff.query("symbol in " + str(list(filtered.index.values))).to_csv('Best_performer_MACD_values.csv', sep=';', decimal=',')

In [ ]:
filtered['delta'] = filtered['Performance'] - filtered['diff']

In [17]:
filtered.sort_values(by='delta', ascending=False).to_csv('best_performers_MACD.csv', sep=';', decimal=',')

In [18]:
fig, ax = plot.subplots(figsize=(10,5))
filtered.sort_values(by='delta', ascending=False).drop(columns='diff').plot(figsize=(10,5), ax=ax)
filtered.sort_values(by='delta', ascending=False)['diff'].plot.bar(figsize=(10,5), ax=ax)
plot.savefig('Performance_comparison.svg')
plot.close()

In [13]:
import pandas as pd
from stockstats import StockDataFrame as Sdf
from tqdm.notebook import tqdm

data = pd.read_pickle('dataset.pkl')
data = data.drop(data.index[0])
filtered = pd.read_csv('best_performers_MACD.csv', sep=';', decimal=',').set_index('Unnamed: 0')

score = []
name = []

for company in filtered.index.values:
    stock  = Sdf.retype(get_stock_data(data, company))

    signal = stock['macds']        # Your signal line
    macd   = stock['macd']         # The MACD that need to cross the signal line
    #                                              to give you a Buy/Sell signal
    listLongShort = ["No data"]    # Since you need at least two days in the for loop

    for i in range(1, len(signal)):
        #                          # If the MACD crosses the signal line upward
        if macd[i] > signal[i] and macd[i - 1] <= signal[i - 1]:
            listLongShort.append(1)
        #                          # The other way around
        elif macd[i] < signal[i] and macd[i - 1] >= signal[i - 1]:
            listLongShort.append(-1)
        #                          # Do nothing if not crossed
        else:
            listLongShort.append(0)

    stock['Advice'] = listLongShort

    # The advice column means "Buy/Sell/Hold" at the end of this day or
    #  at the beginning of the next day, since the market will be closed
    stock['Advice'][0] = 0

    #if stock[stock['Advice']!=0]['advice'][-1] == 1:
    #    stock['Advice'][-1] = -1


    #print(stock[stock['Advice']!=0]['advice'][-1])


    gain = stock['close']*stock['Advice']*(-1)
    stock['gain'] = gain
    #score.append(gain.sum()/(stock['close'][-1]-stock['close'][0]))
    #name.append(symbol)
    stock['Advice']=stock['Advice']*100
    stock.drop(columns=['volume', 'gain']).plot(figsize=(30,10))
    plot.title(str(round(gain.sum()*100/(stock['close'][0]))) + '%')
    plot.savefig(str(company) + '_test.png')
    plot.close()

/home/wonka/.local/lib/python3.10/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
/home/wonka/.local/lib/python3.10/site-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
/tmp/ipykernel_45654/2924528357.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stock['Advice'][0] = 0
/home/wonka/.local/lib/python3.10/site-packag

## Action on current owned actions

In [16]:
import pandas as pd
from stockstats import StockDataFrame as Sdf
from tqdm.notebook import tqdm

data = pd.read_pickle('dataset.pkl')
data = data.drop(data.index[0])
owned = ['NVDA','AMD']

score = []
name = []
advice = {}

for company in owned:
    stock  = Sdf.retype(get_stock_data(data, company))

    signal = stock['macds']        # Your signal line
    macd   = stock['macd']         # The MACD that need to cross the signal line
    #                                              to give you a Buy/Sell signal
    listLongShort = ["No data"]    # Since you need at least two days in the for loop

    for i in range(1, len(signal)):
        #                          # If the MACD crosses the signal line upward
        if macd[i] > signal[i] and macd[i - 1] <= signal[i - 1]:
            listLongShort.append('BUY')
        #                          # The other way around
        elif macd[i] < signal[i] and macd[i - 1] >= signal[i - 1]:
            listLongShort.append('SELL')
        #                          # Do nothing if not crossed
        else:
            listLongShort.append('HOLD')

    stock['Advice'] = listLongShort

    # The advice column means "Buy/Sell/Hold" at the end of this day or
    #  at the beginning of the next day, since the market will be closed
    stock['Advice'][0] = 'HOLD'
    print(stock['Advice'][-1])
    stock.drop(columns=['volume']).plot(figsize=(30,10))
    plot.title(company)
    plot.savefig(str(company) + '_owned.png')
    plot.close()
    advice[company]=stock['Advice'][-1]

pd.DataFrame.from_dict(advice, orient='index').to_markdown()

/home/wonka/.local/lib/python3.10/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
/home/wonka/.local/lib/python3.10/site-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
/tmp/ipykernel_45654/2803904769.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stock['Advice'][0] = 'HOLD'


HOLD


/home/wonka/.local/lib/python3.10/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
/home/wonka/.local/lib/python3.10/site-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
/tmp/ipykernel_45654/2803904769.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stock['Advice'][0] = 'HOLD'


HOLD


In [47]:
import os
os.system("telegram-send "+ "'" + ff + "'")

|      | 0    |
|:-----|:-----|
| NVDA | HOLD |
| AMD  | HOLD |


0

In [3]:
from yahoofinancials import YahooFinancials
import pandas as pd
from loguru import logger

In [4]:
from yahoofinancials import YahooFinancials

yahoo_financials = YahooFinancials(stocks_list['Ticker'].values)
val = yahoo_financials.get_price_to_sales()

In [6]:
#@logger.catch
ff = pd.read_pickle('descriptions.pkl')
#ff = ff[['symbol', 'sector']]
#ff.set_index('symbol', inplace=True)
dt = pd.DataFrame(ff['symbol'])
dt['Industry'] = ff.industry
dt['P/S'] = dt.symbol.map(val)

In [7]:
dt.Industry.replace(0,'unknown', inplace=True)
dt['P/S'].fillna(0, inplace=True)
dt.sort_values('Industry', inplace=True)

In [8]:
dt.to_csv('PS stocks and industry.csv')

In [24]:
dt['P/S'] = pd.to_numeric(dt['P/S'])
for elem in dt.Industry.unique():
    dt[dt['Industry']==elem].sort_values('P/S').plot.bar(x='symbol', y='P/S', figsize=(12, 4)).get_figure().savefig(str(elem) +'.png')
    plot.close()

In [23]:
dt

symbol              Industry        P/S
722   BRKB                         0.000000
562   NVCR                        42.766994
639    OMC  Advertising Agencies   1.265758
281    CCO  Advertising Agencies   0.000000
417   BITA  Advertising Agencies   0.000000
..     ...                   ...        ...
450      S               unknown   0.000000
757    CRM               unknown  10.058805
275   FCAU               unknown   0.000000
518    RTN               unknown   0.000000
92    CRZO               unknown   0.000000

[783 rows x 3 columns]